# Scraping script for obtaining data on data related jobs in Malaysia

In [74]:
import time
#from selenium import webdriver
from bs4 import BeautifulSoup
#from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import re
import os
from lxml import etree
import lxml.html, lxml.html.clean
from urllib.parse import urlparse
from urllib.parse import parse_qs
import pandas as pd
from itertools import product
import time


#driver = webdriver.Chrome(ChromeDriverManager().install())
verbose_flag = True
# htmlSource = driver.page_source

In [6]:
data_engineer_site = r'https://www.jobstreet.com.my/en/job-search/data-engineer-jobs/'
main_site=r"https://www.jobstreet.com.my"

page = requests.get(data_engineer_site)

soup = BeautifulSoup(page.content, 'html.parser')



In [7]:
#find number of result pages
results = soup.find('select', attrs={"id":"pagination"})
options = results.find_all('option')

num_result_pages = int(options[-1]['value'])

In [8]:
jobslist = soup.find('div', attrs={'id':"jobList"}).find('div', attrs={'data-automation':'jobListing'})

In [9]:
jobs = jobslist.find_all('a', attrs={'class':'_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h', 'href': re.compile(r'^\/en\/job\/')})

In [10]:
joblink = jobs[29].get('href')

In [11]:
job_fulllink = main_site + joblink
#job_fulllink = r"https://www.jobstreet.com.my/en/job/data-engineer-4834504?jobId=jobstreet-my-job-4834504&sectionRank=1&token=0~9d2b2714-12eb-4e27-b40c-24f3babd0c52&fr=SRP%20View%20In%20New%20Tab"
job_page = requests.get(job_fulllink)

job_soup = BeautifulSoup(job_page.content, 'html.parser')


In [12]:
parsed_url = urlparse(job_fulllink)
captured_value = parse_qs(parsed_url.query)
print(captured_value['jobId'])

['jobstreet-my-job-4834717']


In [13]:
jobs_dom = etree.HTML(str(job_soup))

In [14]:
job_title = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/h1')[0].text
company_name = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[1]/div/div/div[2]/div/div/div/div[2]/span')[0].text
salary_range = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div/div[2]/span')[0].text

cleaner = lxml.html.clean.Cleaner(allow_tags=[''])
cleaned = cleaner.clean_html(etree.tostring(jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/span/div')[0]))
print()
job_description = cleaned.decode("utf-8").replace('<div>', '').replace('</div>','')
#job_description = etree.tostring(jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/span/div')[0])
# job_level = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/div/div[1]/div/div/div[2]/span')[0].text
# years_experience = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/div/div[3]/div/div/div[2]/span')[0].text
# qualifications = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/div/div[3]/div/div/div[2]/span')[0].text
# job_type = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[2]/div/div[2]/div/div/div[4]/div/div/div[2]/span')[0].text
# company_registration = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div/div[2]/span')[0].text
# company_size = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div/div[2]/span')[0].text
# industry_type = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[2]/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div/div[2]/span')[0].text
# location = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div/div[1]/div/span')[0].text

if verbose_flag:
    print('job: ', job_title)
    print('company: ', company_name)
    print("salary range: ", salary_range)
    print('job description: ', job_description)




job:  Manager, Workday Data & Testing Lead (Contract 1 year)
company:  Sime Darby Plantation Berhad
salary range:  Posted on 26-Feb-22
job description:  Overview of Job SummaryIn this role, you will work in Workday Data &amp; Test Lead capacity owning process of identification of test scenarios, creation of test cases, execution, defect logging / tracking and ensure timely closure of all test phases. The Workday Data &amp; Test Lead will help to move a legacy system to Workday.We are looking for a candidate who has excellent communication skills as you will work with the functional leads and SME’s to determine the data to be moved into Workday. You will communicate conversion solutions across the project, products and the business.Job Description1. Data MigrationProvide technical knowledge and expertise related to current systems eg. Build the data migration programs, lead the data validation, conversion and mapping activities into workday; Lead the identification of process improveme

In [15]:
for job in jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/span/div/div'):
    print(etree.tostring(job))

b'<div><strong>Overview of Job Summary</strong></div>'
b'<div>In this role, you will work in Workday Data &amp; Test Lead capacity owning process of identification of test scenarios, creation of test cases, execution, defect logging / tracking and ensure timely closure of all test phases. The Workday Data &amp; Test Lead will help to move a legacy system to Workday.</div>'
b'<div>We are looking for a candidate who has excellent communication skills as you will work with the functional leads and SME&#8217;s to determine the data to be moved into Workday. You will communicate conversion solutions across the project, products and the business.</div>'
b'<div><strong>Job Description</strong></div>'
b'<div><strong>1. Data Migration</strong></div>'
b'<div><strong>2. Test Design &amp; Management</strong></div>'
b'<div><strong>Education / Professional Qualification</strong></div>'


In [16]:
cleaner = lxml.html.clean.Cleaner(allow_tags=[''])
cleaned = cleaner.clean_html(etree.tostring(jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/span/div/div')[0]))
print(cleaned.decode("utf-8").replace('<div>', '').replace('</div>',''))

Overview of Job Summary


In [17]:
print(etree.tostring(job_description))

TypeError: Type 'str' cannot be serialized.

In [ ]:
job_title = employer_details.find('h1', attrs={'class':'sx2jih0 _18qlyvc0 _18qlyvch _1d0g9qk4 _18qlyvcp _18qlyvc1x'}).text
print(job_title)
company = employer_details.find('span', attrs={'class':'sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc2 _1d0g9qk4 _18qlyvcb'}).text
print(company)
salary_range = employer_details.findAll('span', attrs={'class':'sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _18qlyvca'}).
print(salary_range)

Big Data & Analytics Developer
UCSI University
[<span class="sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _18qlyvca">Kuala Lumpur</span>, <span class="sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _18qlyvca">MYR 5,000 - MYR 6,500</span>, <span class="sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _18qlyvca">Posted on 11-Feb-22</span>]


# Definition of scraping functions

In [115]:
def search(keyword:str, region:str) -> list:
    '''
        search jobstreet for keyword, returns results as a list of job results
    '''
    search_term = keyword.replace(' ', '-')
    root_link = r'https://www.jobstreet.com.my/en/job-search/'
    search_link = root_link + search_term + '-jobs' + "-in-" + region
    page = requests.get(search_link)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find('select', attrs={"id":"pagination"})
    if(results is not None):
        options = results.find_all('option')
        num_result_pages = int(options[-1]['value'])
    else:
        num_result_pages = 1
    print(f"searching @ {search_link}")
    print(f"total results pages for search {keyword}:", {str(num_result_pages)})

    all_results = []
    print("scraping page 1...")
    first_page_results = get_job_links(search_link)
    all_results = all_results + first_page_results

    #get the rest of the result pages
    if(num_result_pages > 1):
        for page in range(2, num_result_pages+1):
            print(f"scraping page {page}...")
            search_page = search_link + str(page)
            results_job_links = get_job_links(search_page)
            all_results = all_results + results_job_links

    print("done!")
    return(all_results)
    


def get_job_links(page_link:str) -> list:
    results_job_links = []
    page = requests.get(page_link)
    soup = BeautifulSoup(page.content, 'html.parser')
    jobs_list = soup.find('div', attrs={'id':"jobList"}).find('div', attrs={'data-automation':'jobListing'})
    jobs = jobs_list.find_all('a', attrs={'class':'_1hr6tkx5 _1hr6tkx8 _1hr6tkxb sx2jih0 sx2jihf zcydq8h', 'href': re.compile(r'^\/en\/job\/')})
    for job in jobs:
        job_link = job.get('href')
        results_job_links.append(job_link)

    return results_job_links


def extract_job_details(job_page_link:str)->dict:
    try:
        root_link = r"https://www.jobstreet.com.my"
        full_link = root_link + job_page_link
        job_page = requests.get(full_link)
        if(job_page.status_code != 200):
            time.sleep(2)
            return extract_job_details(job_page_link)
        job_soup = BeautifulSoup(job_page.content, 'html.parser')
        parsed_url = urlparse(full_link)
        captured_value = parse_qs(parsed_url.query)
        
        jobID = captured_value['jobId']
        
        jobs_dom = etree.HTML(str(job_soup))
        job_title = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[1]/div/div/div[2]/div/div/div/div[1]/h1')[0].text
        salary_range = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[2]/div/div/div/div[2]/span')[0].text

        cleaner = lxml.html.clean.Cleaner(allow_tags=[''])
        cleaned = cleaner.clean_html(etree.tostring(jobs_dom.xpath('//*[@id="contentContainer"]/div/div[2]/div/div[1]/div')[0]))
                                                                    #//*[@id="contentContainer"]/div/div[2]/div/div[1]/div/div[1]/div/div[2]/div/span/div
        job_description = cleaned.decode("utf-8").replace('<div>', '').replace('</div>','')
        job_level = job_soup.find('span', string="Career Level")
        if(job_level is not None):
            job_level = list(list(job_level.parent.parent.children)[1].children)[0].text    

        experience = job_soup.find('span', string="Years of Experience")
        if (experience is not None):                 
            experience = list(list(experience.parent.parent.children)[1].children)[0].text    

        job_type = job_soup.find('span', string="Job Type")
        if(job_type is not None):
            job_type = list(list(job_type.parent.parent.children)[1].children)[0].text 

        qualifications = job_soup.find('span', string="Qualification")
        if(qualifications is not None):
            qualifications = list(list(qualifications.parent.parent.children)[1].children)[0].text

        job_specialization = job_soup.find('span', string="Job Specializations")
        if(job_specialization is not None):
            job_specialization = list(list(job_specialization.parent.parent.children)[1].children)[0].text


        company_name = jobs_dom.xpath('//*[@id="contentContainer"]/div/div[1]/div[1]/div[1]/div/div/div[1]/div/div/div[2]/div/div/div/div[2]/span')[0].text
        company_registration = job_soup.find('span', string="Registration No.")
        if(company_registration is not None):
            company_registration = list(list(company_registration.parent.parent.children)[1].children)[0].text

        company_size = job_soup.find('span', string="Company Size")
        if(company_size is not None):
            company_size = list(list(company_size.parent.parent.children)[1].children)[0].text

        company_industry = job_soup.find('span', string="Industry")
        if(company_industry is not None):
            company_industry = list(list(list(company_industry.parent.parent.children)[1].children)[0].children)[0].text
        
        return {"job_id":jobID, 'job_title': job_title, "salary_range":salary_range, "job_description":job_description, "job_level":job_level, "experience":experience, "job_type":job_type, "qualifications":qualifications, 'job_specialization':job_specialization, 'company_name':company_name, 'company_registration':company_registration, "company_size":company_size, "company_industry":company_industry}
    
    except Exception as e:
        print('exception caught!', str(e))
        print(job_page.status_code)
        return {"job_id":None, 'job_title': None, "salary_range":None, "job_description":None, "job_level":None, "experience":None, "job_type":None, "qualifications":None, 'job_specialization':None, 'company_name':None, 'company_registration':None, "company_size":None, "company_industry":None}

def get_job_page(job_page_link:str):
    root_link = r"https://www.jobstreet.com.my"
    full_link = root_link + job_page_link
    job_page = requests.get(full_link)
    job_soup = BeautifulSoup(job_page.content, 'html.parser')
    parsed_url = urlparse(job_fulllink)
    captured_value = parse_qs(parsed_url.query)
    
    job_ID = captured_value['jobId']

    return {'job_ID': job_ID, 'page':page.content}
    


In [19]:
df = pd.DataFrame(columns=['a', 'b', 'c'])

data1 = {'a':1, 'b':2, 'c':3}
data2 = {'a':4, 'b':5, 'c':6}

df = df.append(data1, ignore_index=True)
df = df.append(data2, ignore_index=True)
df

C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\518163943.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data1, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\518163943.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data2, ignore_index=True)


,a,b,c
0,1,2,3
1,4,5,6


In [120]:
states = ["negeri-sembilan", "kedah", "pahang", "perak", "selangor", "sabah", "johor", "sarawak", "terengganu", "perlis", "kelantan", "melaka", "penang", "kuala-lumpur"]
#states = ["negeri-sembilan"]
job_search_terms = ["data engineer", "data scientist", "data analyst"]
#job_search_terms = ['data engineer']

columns = ["job_id", 'job_title', "salary_range", "job_description", "job_level", "experience", "job_type", "qualifications", 'job_specialization', 'company_name', 'company_registration', "company_size", "company_industry", "job_link", "search_term", "state"]
jobs_df = pd.DataFrame(columns=columns)


for search_term, state in product(job_search_terms, states):
    print(search_term, state)
    results = search(search_term, state)

    for job_link in results:
        time.sleep(0.2)
        job_details = extract_job_details(job_link)
        job_details['job_link'] = job_link
        job_details['search_term'] = search_term
        job_details['state'] = state

        jobs_df = jobs_df.append(job_details, ignore_index=True)


data engineer negeri-sembilan
searching @ https://www.jobstreet.com.my/en/job-search/data-engineer-jobs-in-negeri-sembilan
total results pages for search data engineer: {'1'}
scraping page 1...
done!


C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df

data engineer kedah
searching @ https://www.jobstreet.com.my/en/job-search/data-engineer-jobs-in-kedah
total results pages for search data engineer: {'1'}
scraping page 1...
done!


C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df

data engineer pahang
searching @ https://www.jobstreet.com.my/en/job-search/data-engineer-jobs-in-pahang
total results pages for search data engineer: {'1'}
scraping page 1...
done!


...

data analyst negeri-sembilan
searching @ https://www.jobstreet.com.my/en/job-search/data-analyst-jobs-in-negeri-sembilan
total results pages for search data analyst: {'3'}
scraping page 1...
scraping page 2...
scraping page 3...
done!


C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df

data analyst kedah
searching @ https://www.jobstreet.com.my/en/job-search/data-analyst-jobs-in-kedah
total results pages for search data analyst: {'5'}
scraping page 1...
scraping page 2...
scraping page 3...
scraping page 4...
scraping page 5...
done!


C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df.append(job_details, ignore_index=True)
C:\Users\RSYeo\AppData\Local\Temp\ipykernel_16772\3319905659.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_df = jobs_df

KeyboardInterrupt: 

In [123]:
path = r"C:\Users\RSYeo\Projects\data\temp_scrape.ftr"
jobs_df.to_feather(path)

In [119]:
jobs_df.iloc[2]

job_id                                         [jobstreet-my-job-4828580]
job_title                                          Project/Civil Engineer
salary_range                                          Posted on 13-Feb-22
job_description         Job HighlightsCareer DevelopmentMultinational ...
job_level                                                Junior Executive
experience                                                           None
job_type                                                         Contract
qualifications          Diploma, Advanced/Higher/Graduate Diploma, Bac...
job_specialization              Building/Construction, Civil/Construction
company_name                                      HSL CONSTRUCTOR SDN BHD
company_registration                                             853684-U
company_size                                          201 - 500 Employees
company_industry                    Construction / Building / Engineering
job_link                /en/job/projec

In [ ]:
JD = extract_job_details(results[1])
JD2 = extract_job_details(results[3])


In [ ]:
df = pd.DataFrame(JD)

In [ ]:
df = df.append(JD2, ignore_index=True)

In [26]:
df.head(50)

,a,b,c
0,1,2,3
1,4,5,6


In [ ]:
page = requests.get(r"https://www.jobstreet.com.my/en/job/assistant-vice-president-data-engineer-compliance-amlt-remediation-4824511?fr=SRP%20Job%20Listing&jobId=jobstreet-my-job-4824511&sectionRank=2&token=0~2cdf76e9-a859-4c15-bb9a-e16a948b507a")

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')
list(list(soup.find('span', string="Career Level").parent.parent.children)[1].children)[0].text

'Manager'

In [97]:
extract_job_details('/en/job/data-scientist-9316926/origin/sg?jobId=jobstreet-sg-job-9316926&sectionRank=30&token=0~474bcf85-700c-437e-8294-235da1e78670&fr=SRP%20Job%20Listing')

{'job_id': ['jobstreet-sg-job-9316926'],
 'job_title': 'Data Scientist',
 'salary_range': 'MYR\xa010,000 - MYR\xa018,000',
 'job_description': "Job DescriptionResponsibilities:Responsible for designing and implementing solutions for complex systems, large-scale data sets, data mining, and research purposes.Data model creation, development, visualization &amp; deploymentDevelop machine learning algorithms to address technology requirements and gapsDevelop &amp; deploy predictive models with internal big &amp; small data setsCollaborate with the Data Engineering, Product, Business &amp; Operations teamsRequirements:Bachelor degree in Computer Science, Engineering, Statistics or Mathematics or similar fieldGood understanding of and experience with machine learning models and data analysisProficiency in Python/Matlab/ /RExperience with Tensorflow, PyTorch scikit-learn or tensorflowThe above information on this description has been designed to indicate the general nature, and level, of the 